In [1]:
using Nemo, ExactWrightFisher


Welcome to Nemo version 0.11.0

Nemo comes with absolutely no warranty whatsoever



In [2]:
RR = RealField(256)

Real Field with 256 bits of precision and error bounds

In [3]:
RR(3)

3.00000000000000000000000000000000000000000000000000000000000000000000000000000

In [4]:
import Base.sign

function sign(x::arb)
    if x < 0
        return -1
    else
        return 1
    end
end

sign (generic function with 10 methods)

In [5]:
function signed_logsumexp_arb(lx, signs)
    res = sum(exp.(RR.(lx)) .* signs)
    return sign(res), log(abs(res))
    
end
function signed_logsumexp_naive(lx, signs)
# There might potentially be a problem when the larger terms cancel each other
  m = maximum(lx)
  scaled_sum = sum(signs .* exp.(lx .- m))
  if abs(scaled_sum) <= 10*eps(Float64)
    return [1., -Inf]
  elseif scaled_sum < 0
    sgn = -1
    scaled_sum = -1*scaled_sum
  else
    sgn = 1
  end
  return [sgn, m + log(scaled_sum) ]#Will give an error if the sum is negative
end    

signed_logsumexp_naive (generic function with 1 method)

In [6]:
ExactWrightFisher.signed_logsumexp([0.2, -3., 1], [1, 1, -1])[2]

0.36955602678296184

In [7]:
signed_logsumexp_arb([0.2, -3., 1], [1, 1, -1])

(-1, [0.369556026782961808705575872199613349049838666327407972371910546740124422446 +/- 3.98e-76])

In [8]:
signed_logsumexp_naive([0.2, -3., 1], [1, 1, -1])[2]

0.36955602678296184

In [9]:
ExactWrightFisher.S_kvec_M_both_logsumexp([1,3,50, 10], 0.0002, 0.075)
function S_kvec_M_plus_logsum_test(kvec::Array{T, 1}, t::Real, θ::Real, logsumexpfun) where
T<:Integer
  M = length(kvec)

  two_kvec_plus_1 = sum(2*kvec .+ 1)

  U = typeof(t)

  logterms = Array{U}(undef, two_kvec_plus_1)
  signs = Array{Float64}(undef, two_kvec_plus_1)
  cnt = 1
  for m in 0:(M-1)
    for i in 0:(2*kvec[m+1])
    # for(int i = 0; i <= 2*kvec[m]; ++i) {
      logterms[cnt] = ExactWrightFisher.log_bk_t_θ_t(m+i, t, θ, m);
      signs[cnt] = ExactWrightFisher.minus_1_power_i(i);
      cnt += 1;
    end
  end
  return logsumexpfun(logterms, signs);
end
S_kvec_M_plus_logsum_arb(kvec::Array{T, 1}, t::Real, θ::Real) where
T<:Integer = S_kvec_M_plus_logsum_test(kvec, t, θ, signed_logsumexp_arb)
S_kvec_M_plus_logsum_naive(kvec::Array{T, 1}, t::Real, θ::Real) where
T<:Integer = S_kvec_M_plus_logsum_test(kvec, t, θ, signed_logsumexp_naive)

S_kvec_M_plus_logsum_naive (generic function with 1 method)

In [10]:
ExactWrightFisher.S_kvec_M_plus_logsum([1,3,50, 10], 0.0002, 0.075)  .- S_kvec_M_plus_logsum_arb([1,3,50, 10], 0.0002, 0.075)

2-element Array{Any,1}:
 0.0                                                                               
  [3.259499352081579912276251678228916106881957897961454176215791e-14 +/- 2.10e-75]

In [11]:
S_kvec_M_plus_logsum_naive([1,3,50, 10], 0.0002, 0.075) .- S_kvec_M_plus_logsum_arb([1,3,50, 10], 0.0002, 0.075)

2-element Array{Any,1}:
 0.0                                                                             
  [6.2057041161129075856192474127353606881957897961454176215791e-16 +/- 2.10e-75]

In [12]:
ExactWrightFisher.S_kvec_M_plus_logsum([1,3,50, 10], 0.0002 |> BigFloat, 0.075 |> BigFloat) .- S_kvec_M_plus_logsum_arb([1,3,50, 10], 0.0002, 0.075)

2-element Array{Any,1}:
 0.0                                                                                
  [4.4501330007744136061580208744544390503666193058028739165565813e-14 +/- 9.00e-76]

In [13]:
S_kvec_M_plus_logsum_naive([1,3,50, 10], 0.0002 |> BigFloat, 0.075 |> BigFloat) .- S_kvec_M_plus_logsum_arb([1,3,50, 10], 0.0002, 0.075)

2-element Array{Any,1}:
 0.0                                                                                
  [4.4501330007744136061580208744544390503666193058028739165565816e-14 +/- 7.04e-76]

In [14]:

function Compute_A∞_given_U_debug(θ, t, U, m, kvec; start_print_from = 32)
  ### 0 indexing to stick with the article's notation
    n=0
  while true
#     n = n+1
#     print(n)
#     if (n > 100){
#       return(m)
# #       break()
#     }
        if m >= start_print_from
            println(m)
        end
        kvec[m+1] = ceil(ExactWrightFisher.C_m_t_θ(m, t, θ)/2)
        #     print(km)
        S_kvec_M_BOTH = ExactWrightFisher.S_kvec_M_both_logsumexp(kvec, t, θ)
    while (S_kvec_M_BOTH[1] < U) && (S_kvec_M_BOTH[2] > U)
        kvec = kvec .+ 1
            #       println(kvec)
        S_kvec_M_BOTH = ExactWrightFisher.S_kvec_M_both_logsumexp(kvec, t, θ)
        if m >= start_print_from
            println(S_kvec_M_BOTH)
        end
    end
    if S_kvec_M_BOTH[1] > U
      println("A∞ = $m")
      return m
      # break()
    elseif (S_kvec_M_BOTH[2] < U)
      push!(kvec,0)
      m = m + 1
    end
  end
end

function Compute_A∞_given_U_arb_debug(θ, t, U, m, kvec; start_print_from = 32)
  ### 0 indexing to stick with the article's notation
    n=0
  while true
#     n = n+1
#     print(n)
#     if (n > 100){
#       return(m)
# #       break()
#     }
        if m >= start_print_from
            println(m)
        end
        kvec[m+1] = ceil(ExactWrightFisher.C_m_t_θ(m, t, θ)/2)
        #     print(km)
        S_kvec_M_BOTH = ExactWrightFisher.S_kvec_M_both_logsumexp_arb(kvec, t, θ)
    while (S_kvec_M_BOTH[1] < U) && (S_kvec_M_BOTH[2] > U)
        kvec = kvec .+ 1
        #       println(kvec)
        S_kvec_M_BOTH = ExactWrightFisher.S_kvec_M_both_logsumexp_arb(kvec, t, θ)
        if m >= start_print_from
            println(S_kvec_M_BOTH)
        end
    end
    if S_kvec_M_BOTH[1] > U
      println("A∞ = $m")
      return m
      # break()
    elseif (S_kvec_M_BOTH[2] < U)
      push!(kvec,0)
      m = m + 1
    end
  end
end

Compute_A∞_given_U_arb_debug (generic function with 1 method)

In [17]:
Compute_A∞_given_U_debug(sum(1:4), 0.05, 0.6, 0, [0])

32
[-2.28284e9, 3.35367e9]
[-7.3275e8, 1.37184e9]
[-1.50113e8, 3.49899e8]
[-2.03743e7, 5.81103e7]
[-1.88515e6, 6.49163e6]
[-1.2161e5, 5.00275e5]
[-5569.14, 27137.6]
[-183.255, 1053.85]
[-3.85921, 30.2198]
[0.488993, 1.18006]
[0.605274, 0.615684]
A∞ = 32


32

In [18]:
Compute_A∞_given_U_arb_debug(sum(1:4), 0.05, 0.6, 0, [0])

32
arb[[-2282844386.799139298066442987490941845084257320419956114716521507888598957 +/- 6.65e-64], [3353666667.110587364797343551455478348672247378285112485799754351004289311 +/- 8.76e-64]]
arb[[-732750018.756811365873079054952211803336851024779414841682882220623356863 +/- 4.02e-64], [1371839138.212141384984172295327439135559446786948466886858392846056775100 +/- 3.94e-64]]
arb[[-150113158.617359828518736583856202151358466008537851581456652708975762086 +/- 4.87e-64], [349899079.579565153213944531556430984444116260391945943306289294932891311 +/- 7.92e-64]]
arb[[-20374344.469023653482302586961126730768700328121189440011911529226822644 +/- 4.06e-64], [58110342.763841847284811985701007708386202227479844040104977065100928678 +/- 5.74e-64]]
arb[[-1885150.548302914397567180418648252732242725705558540320696031854982656 +/- 6.70e-64], [6491627.381429476755036376935665340156819449607273210420529991344189875 +/- 6.45e-64]]
arb[[-121609.84741363445770441151443618883449895464484521350038886475975209

arb[[-0.493536771492650030741268463864684674237706905677946559816552071 +/- 9.04e-64], [8.446261660441881256480709964694245134902664196905564288345632017 +/- 8.07e-64]]
arb[[0.678650072459080945256182715432567003960524390247582862677798124 +/- 5.47e-64], [0.871262553646108274575457653536506760745667334273473563281740709 +/- 6.90e-64]]
A∞ = 36


36